In [1]:
import Rust_Machine_Learning
import plotly.express as px
import pandas as pd
from syed_ml_lib import *

In [2]:
# Get the csv data
df = pd.read_csv("../data/iris.data")
# df = pd.read_csv("../data/kmeans-dataset.csv")
# df = pd.read_csv("../data/winequality-red.csv")
# df = pd.DataFrame(data={
#   "chest_pain": [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0],
#   "blocked_arteries": [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0],
#   "patient_weight": [205, 180, 210, 167, 156, 125, 168, 172],
#   "heart_disease": [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
# })
df  

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
to_encode = [4]
encoded, feature_to_encoded_cols = one_hot_encode(df, to_encode)
encoded

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0


In [4]:
# Prepare data for training with input rust functions expect
label_col = 1

train_data, test_data = split_train_test(encoded, 0.75, True)

# Remove any encoded data of the same class
train_data_features = train_data.copy()
remove_encoded_category_data_and_label(train_data_features, feature_to_encoded_cols, label_col)
train_data_features_list = train_data_features.values.tolist()

# Get labels
train_data_labels = train_data.iloc[:, label_col].values.tolist()

# Determine which columns in train_data_features_list are categorical
is_categorical = [False, False, False, False]

train_data_features

,sepal length (cm),petal length (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
120,6.9,5.7,2.3,0.0,0.0,1.0
42,4.4,1.3,0.2,1.0,0.0,0.0
144,6.7,5.7,2.5,0.0,0.0,1.0
31,5.4,1.5,0.4,1.0,0.0,0.0
79,5.7,3.5,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...
35,5.0,1.2,0.2,1.0,0.0,0.0
5,5.4,1.7,0.4,1.0,0.0,0.0
8,4.4,1.4,0.2,1.0,0.0,0.0
91,6.1,4.6,1.4,0.0,1.0,0.0


In [5]:
# Now train the model
tree_depth = 10
forest_size = 100
learning_rate = 0.5
is_categorical = False
trained_model = Rust_Machine_Learning.GradientBoost(
    train_data_features_list, train_data_labels, tree_depth, forest_size, learning_rate, is_categorical
)

# trained_model.print()

In [6]:
# Now let us test the model
test_data_features = test_data.copy()
remove_encoded_category_data_and_label(test_data_features, feature_to_encoded_cols, label_col)
test_data_features = test_data_features.values.tolist()
test_data_labels = test_data[test_data.columns[label_col]].values.tolist()

labelled_results = trained_model.classify(test_data_features)

num_correct = 0
tolerance = 0.10
for test_label, result_label in zip(test_data_labels, labelled_results):
  num_correct += (abs(test_label-result_label)/test_label <= tolerance)
print("Percent Correct: ", num_correct / len(test_data_labels))
labelled_results

Percent Correct:  0.7368421052631579


[3.131474574404486,
 3.08619609852506,
 3.0977526612212234,
 3.3769743157492518,
 2.827305124048192,
 2.622120984338911,
 2.933098159365539,
 3.612260955277303,
 2.7982495193360437,
 2.9662121073310206,
 2.697902328780389,
 3.1799000446450036,
 2.7146612937277617,
 3.6768699370661304,
 3.494109482292258,
 3.745399307123065,
 3.3887230601509573,
 2.8714631043809598,
 2.8595848572486995,
 3.1883243489593247,
 2.971477824483751,
 3.024808069043897,
 3.277245910656773,
 3.209241649067239,
 2.7095430289432505,
 3.025864512612868,
 3.28937885868803,
 2.531665591412084,
 3.01524052582022,
 2.3361069425239984,
 3.2156685154046576,
 3.760559934730684,
 2.8449765819568418,
 3.063237819344078,
 3.2879271216519697,
 3.0284557439832964,
 3.6678695114338367,
 3.1158914228510626]